In [ ]:
import sqlite3  # sqlite3 모듈 임포트
import pandas as pd  # pandas 모듈 임포트

## Create Database

In [ ]:
# 엑셀 파일 경로
excel_file = 'w1m2_sqltut.xlsx'
# 데이터베이스 파일 경로
database_file = 'df3.db'
# 엑셀 파일을 판다스 데이터프레임으로 읽기 (모든 시트)
xls = pd.ExcelFile(excel_file)
# SQLite 데이터베이스 연결 생성
conn = sqlite3.connect(database_file)
# 각 시트를 반복하면서 데이터베이스에 테이블로 저장
for sheet_name in xls.sheet_names:
    # 각 시트를 데이터프레임으로 변환
    df = pd.read_excel(xls, sheet_name=sheet_name)
    # 각 컬럼별로 map 함수를 사용하여 변환
    for col in df.columns:
        if df[col].dtype == 'object':  # 문자열 데이터 타입 확인
            df[col] = df[col].map(lambda x: x.replace('\xa0', '').replace('\xa9', '') if isinstance(x, str) else x)
    # 데이터프레임을 SQLite 데이터베이스 테이블로 저장
    df.to_sql(sheet_name, conn, if_exists='replace', index=False)

In [ ]:
db_file = 'df3.db'

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

## SELECT

In [ ]:
def query(cursor, query: str):
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)

In [ ]:
query(cursor, "SELECT * FROM Customers")

In [ ]:
query(cursor, "SELECT CustomerName, City FROM Customers")

In [ ]:
query(cursor, "SELECT DISTINCT Country FROM Customers")

In [ ]:
query(cursor, "SELECT COUNT(DISTINCT Country) FROM Customers")

In [ ]:
# Numeric field에서는 single quotes를 사용하지 않아도 됨.
# Text field에서는 double quotes와 혼동하지 않도록 single quotes를 사용해야 함.

query(cursor, "SELECT * FROM Customers WHERE Country='Mexico'")

In [ ]:
query(cursor, "SELECT * FROM Customers WHERE CustomerID > 80;")

In [ ]:
# ORDER BY 쿼리 실행
query(cursor, "SELECT * FROM Products ORDER BY Price;")

In [ ]:
# Numeric Type에서는 숫자의 오름/내림차순으로 정렬 가능
# Text Type에서는 알파벳의 사전순/역순으로 정렬 가능
# ORDER BY는 중첩해서 사용할 수 있음.

query(cursor, "SELECT * FROM Products ORDER BY Price DESC;")

In [ ]:
query(cursor, "SELECT * FROM Customers ORDER BY Country ASC, CustomerName DESC;")

In [ ]:
query(cursor, "SELECT * FROM Customers WHERE Country='Spain' AND CustomerName Like 'G%';")

In [ ]:
query(cursor, "SELECT * FROM Customers WHERE Country='Germany' AND City='Berlin' AND PostalCode > 12000;")

In [ ]:
query(
    cursor, 
    """
    SELECT * 
    FROM Customers
    WHERE Country='Spain' AND (CustomerName LIKE 'G%' or CustomerName LIKE 'R%');      
    """
)

In [ ]:
# 괄호를 작성하지 않은 경우에는 AND -> OR 순으로 쿼리가 진행됨
# Spain이면서 G로 시작하는 사람이거나, R로 시작하는 사람을 리턴하는 쿼리

query(
    cursor, 
    """
    SELECT * 
    FROM Customers
    WHERE Country='Spain' AND CustomerName LIKE 'G%' or CustomerName LIKE 'R%'; 
    """
)

In [ ]:
query(
    cursor, 
    """
    SELECT * 
    FROM Customers 
    WHERE Country IN ('Germany', 'Spain');
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT * 
    FROM Customers
    WHERE City = 'Berlin' OR CustomerName LIKE 'G%' OR Country = 'Norway';
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT * 
    FROM Customers
    WHERE NOT Country='Spain';
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT * 
    FROM Customers
    WHERE CustomerID NOT BETWEEN 10 AND 60;
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT * 
    FROM Customers
    WHERE City NOT IN ('Paris', 'London');
    """
)

## INSERT

- 엑셀에서 추출한 DB에서는 CustomerID에 대한 AutoIncrement가 빠져있기 때문에 INSERT 명령어를 사용할 때 에러가 발생함
- 새로운 DB를 생성하고 원본 DB 내 row들을 새 DB에 옮겨주는 작업 진행

In [ ]:
# 기존 테이블 데이터 백업
cursor.execute("SELECT * FROM Customers")
rows = cursor.fetchall()

# 기존 테이블 구조 확인
cursor.execute("PRAGMA table_info(Customers);")
columns = cursor.fetchall()
print("Existing Customers table columns:")
for column in columns:
    print(column)

print("\nExisting Customers table data:")
for row in rows:
    print(row)

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Customers_new (
        CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
        CustomerName TEXT NOT NULL,
        ContactName TEXT,
        Address TEXT,
        City TEXT,
        PostalCode TEXT,
        Country TEXT
    )
""")
conn.commit()

In [ ]:
for row in rows:
    cursor.execute("""
        INSERT INTO Customers_new (CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (int(row[0]), row[1], row[2], row[3], row[4], row[5], row[6])
    )
conn.commit()

In [ ]:
cursor.execute("DROP TABLE Customers")
conn.commit()

In [ ]:
cursor.execute("ALTER TABLE Customers_new RENAME TO Customers")
conn.commit()

In [ ]:
cursor.execute("""
    INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
    VALUES ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway');
""")

query(cursor, "SELECT * FROM Customers WHERE CustomerName='Cardinal';")

In [ ]:
cursor.execute("""
    INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
    VALUES
    ('Greasy Burger', 'Per Olsen', 'Gateveien 15', 'Sandnes', '4306', 'Norway'),
    ('Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK');            
""")

query(cursor, "SELECT * FROM Customers ORDER BY CustomerID DESC LIMIT 3;")

In [ ]:
# NULL값은 말그대로 데이터가 존재하지 않는 상태를 의미하며, 비교/동등 연산자를 통해 쿼리할 수 없음.
# IS NULL, IS NOT NULL을 통해 NULL값을 쿼리할 수 있음.

query(
    cursor,
    """
    SELECT CustomerName, ContactName, Address 
    FROM Customers
    WHERE Address IS NULL;
    """
)

## Update

In [ ]:
query(
    cursor,
    """
    UPDATE Customers
    SET ContactName='Alfred Schmidt', City='Frankfurt'
    WHERE CustomerID=1;
    """
)

In [ ]:
# Update에서 Where 절을 작성하지 않으면 모든 행이 업데이트 된다는 것을 주의해야 함.

query(
    cursor,
    """
    UPDATE Customers
    SET ContactName='Juan'
    WHERE Country='Mexico';
    """
)

## Delete

In [ ]:
# Delete에서 Where 절을 작성하지 않으면 모든 행이 삭제된다는 것을 주의해야 함.
# Drop Table은 테이블 자체를 삭제하는 것이므로 주의해야 함. (둘이 다름)

query(
    cursor,
    """
    DELETE FROM Customers
    Where CustomerName='Alfreds Futterkiste';
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT * FROM Customers LIMIT 3;
    """
)

print()
query(
    cursor,
    """
    SELECT * FROM Customers 
    WHERE Country='Germany'
    LIMIT 3;
 """
)

## Aggregation Function

In [ ]:
query(
    cursor,
    """
    SELECT MIN(Price) AS MinPrice, MAX(Price) AS MaxPrice, AVG(Price) AS AvgPrice
    FROM Products;
    """
)

query(
    cursor,
    """
    SELECT CategoryID, MIN(Price) AS MinPrice
    FROM Products
    GROUP BY CategoryID;
    """
)

In [ ]:
# COUNT 함수는 특정 컬럼의 값이 NULL인 경우는 제외하고 카운트함.

query(
    cursor,
    """
    SELECT COUNT(*) AS TotalCount
    FROM Products;
    """
)

In [ ]:
# Alias를 여러 word로 사용할 경우에는 대괄호로 묶어줘야 함.

query(
    cursor,
    """
    SELECT COUNT(*) AS [Number of records], CategoryID
    FROM Products
    GROUP BY CategoryID;
    """
)

In [ ]:
query(
    cursor, 
    """
    SELECT SUM(Quantity) AS TotalQuantity
    FROM OrderDetails
    WHERE ProductID=11;
    """
)
print()

query(
    cursor,
    """
    SELECT OrderID, SUM(Quantity) AS TotalQuantity
    FROM OrderDetails
    GROUP BY OrderID;
    """
)

In [ ]:
# LEFT JOIN

query(
    cursor,
    """
    SELECT SUM(Price * Quantity) AS TotalPrice
    FROM OrderDetails
    LEFT JOIN Products ON OrderDetails.ProductID = Products.ProductID;
    """
)

In [ ]:
# NULL값은 자동으로 무시되어 처리됨.
# Subquery를 사용하여 처리할 수 있음.

query(
    cursor,
    """
    SELECT AVG(Price) AS AvgPrice
    FROM Products;
    """
)
print()

query(
    cursor,
    """
    SELECT * FROM Products
    WHERE price > (SELECT AVG(Price) FROM Products);
    """
)

query(
    cursor,
    """
    SELECT AVG(Price) AS AvgPrice, CategoryID
    FROM Products
    GROUP BY CategoryID;
    """
)

In [ ]:
# LIKE 조건에 있는 영문자의 경우, 대/소문자를 구분하지 않음.
# %는 0개 이상의 문자를 의미하며, _는 1개의 문자를 의미함. (a% => startswith(a)와 동일)
# %ab% => contains(ab)와 동일

query(
    cursor,
    """
    SELECT *
    FROM Customers 
    WHERE CustomerName LIKE 'a%';
    """
)

query(
    cursor,
    """
    SELECT * 
    FROM Customers
    WHERE city LIKE 'L_nd__';
    """
)

query(
    cursor, 
    """
    SELECT *
    FROM Customers
    WHERE CustomerName LIKE '%a';
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT * 
    FROM Customers
    WHERE CustomerName LIKE '[a-c]%';
    """
)

In [ ]:
# IN 구분은 여러 값들을 OR로 묶어서 처리할 때 사용함. (OR로 대체 가능하나, 코드 가독성을 위해 사용)

query(
    cursor,
    """
    SELECT *
    FROM Customers
    WHERE Country IN ('Germany', 'France', 'UK');
    """
)

query(
    cursor,
    """
    SELECT *
    FROM Customers
    WHERE CustomerID IN (SELECT CustomerID FROM Orders);
    """
)

In [ ]:
# BETWEEN은 시작값과 끝값을 포함하여 처리함. (>=, <=와 동일)
# BETWEEN 할 값이 Text Type인 경우, 알파벳 순서로 처리함.

query(
    cursor,
    """
    SELECT * 
    FROM Products
    WHERE Price BETWEEN 10 AND 20;
    """
)

query(
    cursor,
    """
    SELECT *
    FROM Products
    WHERE ProductName BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Govanni'
    ORDER BY ProductName;
    """
)

query(
    cursor,
    """
    SELECT * 
    FROM Orders
    WHERE OrderDate BETWEEN '1996-07-01' AND '1996-07-31';
    """
)

In [ ]:
# Alias는 테이블이나 컬럼에 대한 별칭을 지정할 때 사용함.
# 특히, AGGREGATE FUNCTION을 사용할 때, Alias를 사용하여 컬럼명을 지정할 수 있음.
# Alias에서 multiple words를 사용할 경우, 대괄호 혹은 Double quotes로 묶어줘야 함.

query(
    cursor,
    """
    SELECT CustomerID AS ID, CustomerName AS Customer
    FROM Customers;
    """
)


In [ ]:
# Concatename columns를 진행할 때 mysql은 CONCAT 함수를 사용함
# 기본은 +임
# Table 또한 Alias를 사용하여 조인할 때 편리하게 사용가능함.

query(
    cursor,
    """
    SELECT CustomerName, Address + ', ' + PostalCode + ' ' + City + ', ' + Country AS Address
    FROM Customers;
    """
)

## Join

In [ ]:
# Join은 두 개 이상의 테이블을 연결하여 데이터를 가져오는 방법임.
# 그냥 JOIN을 사용하면 INNER JOIN과 동일하게 작동함.

query(
    cursor,
    """
    SELECT Orders.OrderID, Customers.CustomerName, Orders.OrderDate
    FROM Orders
    INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID;
    """
)

query(
    cursor,
    """
    SELECT Orders.OrderID, Customers.CustomerName, Shippers.ShipperName
    FROM ((Orders
    INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID)
    INNER JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID);
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT Customers.CustomerName, Orders.OrderID
    FROM Customers
    LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID
    ORDER BY Customers.CustomerName;
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT Orders.OrderID, Employees.LastName, Employees.FirstName
    FROM Orders
    RIGHT JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
    ORDER BY Orders.OrderID;
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT Customers.CustomerName, Orders.OrderID
    FROM Customers
    FULL OUTER JOIN Orders ON Customers.CustomerID = Orders.CustomerID
    ORDER BY Customers.CustomerName;
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT A.CustomerName AS CustomerName1, B.CustomerName AS CustomerName2, A.City
    FROM Customers A, Customers B
    WHERE A.CustomerID <> B.CustomerID
    AND A.City = B.City
    ORDER BY A.City;
    """
)

In [ ]:
# Union은 기본적으로 두 테이블 간 중복을 제거하고 합집합을 구함
# Union All은 중복을 제거하지 않고 합집합을 구함

query(
    cursor,
    """
    SELECT City FROM Customers
    UNION
    SELECT City FROM Suppliers
    ORDER BY City;
    """
)

In [ ]:
query(
    cursor,
    """
    SELECT 'Customer' AS Type, ContactName, City, Country
    FROM Customers
    UNION
    SELECT 'Supplier', ContactName, City, Country
    FROM Suppliers;
    """
)

In [ ]:
# Group By는 특정 컬럼을 기준으로 데이터를 그룹화하며, 보통 Aggregate Function과 함께 사용함.

query(
    cursor,
    """
    SELECT COUNT(CustomerID) AS CustomerCnt, Country
    FROM Customers
    GROUP BY Country
    ORDER BY CustomerCnt DESC;
    """
)

query(
    cursor,
    """
    SELECT s.ShipperName, COUNT(o.OrderID) AS OrderCnt 
    FROM Orders o
    LEFT JOIN Shippers s ON o.shipperID = s.ShipperID
    GROUP BY s.ShipperName
    """
)

In [ ]:
# Where절은 Aggregate Function과 사용되지 않으며, 사용하고 싶다면 Having 절을 사용해야 함.

query(
    cursor,
    """
    SELECT COUNT(CustomerID) AS CustomerCNT, Country
    FROM Customers
    GROUP BY Country
    HAVING COUNT(CustomerID) > 5
    ORDER BY CustomerCNT DESC;
    """
)

query(
    cursor,
    """
        SELECT e.LastName, COuNT(o.OrderID) AS OrderCnt
        FROM (
            Orders o
            INNER JOIN Employees e ON o.EmployeeID = e.EmployeeID
        )
        GROUP BY e.LastName
        HAVING COUNT(o.OrderID) > 10;
    """
)

query(
    cursor,
    """
    SELECT e.LastName, COUNT(o.OrderID) AS OrderCnt 
    FROM Orders o
    INNER JOIN Employees e ON o.EmployeeID=e.EmployeeID
    WHERE LastName='Davolio' OR LastName='Fuller'
    GROUP BY e.LastName
    HAVING COUNT(o.OrderID) > 25;
    """
)

In [ ]:
# Exists는 Subquery의 결과가 존재하는지 확인하는 함수임. (T/F를 반환함)

query(
    cursor,
    """
    SELECT SupplierName
    FROM Suppliers
    WHERE EXISTS (
        SELECT ProductName
        FROM Products
        WHERE Products.SupplierID = Suppliers.SupplierID AND Price < 20    
    );
    """    
)

query(
    cursor,
    """
    SELECT SupplierName
    FROM Suppliers
    WHERE EXISTS (
        SELECT ProductName
        FROM Products
        WHERE Products.SupplierID = Suppliers.SupplierID AND Price == 22   
    );
    """    
)

In [ ]:
# ANY, ALL은 Subquery의 결과가 조건에 부합하는지 확인하는 함수임.
# ANY는 하나만 부합하면 True를 반환하며, ALL은 모두 부합해야 True를 반환함.

query(
    cursor,
    """
    SELECT ProductName
    FROM Products
    WHERE ProductID = ANY
    (SELECT ProductID
    FROM OrderDetails
    WHERE Quantity = 10);
    """
) 


In [ ]:
# SELECT INTO는 새로운 테이블을 생성하고 데이터를 복사하는 방법임.
# SQLite3에서는 INTO 구문을 지원하지 않음.
# 여러개의 테이블을 조인하여 새로운 테이블을 생성할 수 있음.

"""
SELECT col1, col2, ...
INTO newtable [IN externaldb]
FROM oldtable
WHERE condition;
"""

query(
    cursor,
    """
    SELECT * 
    INTO CustomersBackup
    IN 'test.db'
    FROM Customers;
    """
)

In [ ]:
# INSERT INTO SELECT는 기존 테이블에서 데이터를 복사하여 새로운 테이블을 생성하는 방법임.

"""
INSERT INTO table2 (column1, column2, column3, ...)
SELECT column1, column2, column3, ...
FROM table1
WHERE condition;
"""

query(
    cursor,
    """
    INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
    SELECT SupplierName, ContactName, Address, City, PostalCode, Country 
    FROM Suppliers;
    """
)

In [ ]:
# CASE는 조건에 따라 다른 값을 반환하는 함수임.
# SELECT, WHERE, ORDER BY, GROUP BY, HAVING, INSERT, UPDATE, DELETE 등 다양한 구문에서 사용 가능함.

"""
CASE
    WHEN condition1 THEN result1
    WHEN condition2 THEN result2
    WHEN conditionN THEN resultN
    ELSE result
END (AS alias);
"""

query(
    cursor,
    """
    SELECT CustomerName, City, Country
    FROM Customers
    ORDER BY(
        CASE
            WHEN City is NULL THEN Country
            ELSE City
        END
    );
    """
)

In [ ]:
# IFNULL(ISNULL)은 NULL값을 다른 값으로 대체하는 함수임. 
# COALESCE는 인수 중 첫 번째로 NULL이 아닌 값을 반환

"""
IFNULL(expression, alt_value) # MySQL
COALESCE(expression1, expression2, expression3, ...) # Any SQL
NVL(expression, alt_value) # Oracle
"""

query(
    cursor,
    """
    SELECT ProductName, (UnitsInStock + COALESCE(UnitsOnOrder, 0))
    FROM Products;
    """
)

In [ ]:
# Stored Procedure는 SQL 명령문을 저장하고 실행(재사용)할 수 있는 함수임.

"""
CREATE PROCEDURE procedure_name
AS 
SQL Statements
GO;

EXEC procedure_name;
"""

# 다중 Parameter를 사용할 수 있으며, Parameter를 사용할 때는 @를 사용함.
"""
CREATE PROCEDURE SelectAllCustomers @City nvarchar(30), @PostalCode nvarchar(10)
AS
SELECT * FROM Customers WHERE City = @City AND PostalCode = @PostalCode
GO;
"""